# Pipeline Tests


## Local Scheduler Pipeline Test

In [1]:
import os
k8s = False
if k8s:
    SCHEDULER_IP=!kubectl get svc seldon-scheduler -n seldon-mesh -o jsonpath='{.status.loadBalancer.ingress[0].ip}'
    SCHEDULER_IP=SCHEDULER_IP[0]
    MESH_IP=!kubectl get svc seldon-mesh -n seldon-mesh -o jsonpath='{.status.loadBalancer.ingress[0].ip}'
    MESH_URL=MESH_IP[0]
else:
    SCHEDULER_IP="0.0.0.0"
    MESH_URL="0.0.0.0:9000"
os.environ['SCHEDULER_IP'] = SCHEDULER_IP
os.environ['MESH_URL'] = MESH_URL
print("Scheduler ip",SCHEDULER_IP)
print("Mesh URL",MESH_URL)

Scheduler ip 0.0.0.0
Mesh URL 0.0.0.0:9000


To test pipeline status updates run in a separate terminal

```bash
grpcurl -d '{"subscriberName":"foo"}' -plaintext -import-path ../../apis -proto ../../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/SubscribePipelineStatus
```

In [ ]:
!grpcurl -d '{"model":{ \
              "meta":{"name":"iris"},\
              "modelSpec":{"uri":"gs://seldon-models/mlserver/iris",\
                           "requirements":["sklearn"],\
                           "memoryBytes":500},\
              "deploymentSpec":{"replicas":1}}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/LoadModel

In [ ]:
!grpcurl -d '{"name":"mlserver"}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/ServerStatus

In [ ]:
!curl -v http://${MESH_URL}/v2/models/iris/infer -H "Content-Type: application/json" -H "seldon-model: iris"\
        -d '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}'

In [ ]:
!grpcurl -d '{"pipeline":{ \
              "name": "pipeline-iris",\
              "steps": [\
                 {"name":"iris"} \
              ], \
              "output":{"inputs":"iris"}}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/LoadPipeline

In [9]:
!grpcurl -d '{"name":"iris","allVersions":true}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/PipelineStatus

{
  "pipelineName": "iris",
  "versions": [
    {
      "pipeline": {
        "name": "iris",
        "steps": [
          {
            "name": "iris"
          }
        ],
        "output": {
          "inputs": [
            "iris.outputs"
          ]
        },
        "kubernetesMeta": {
          
        }
      },
      "state": {
        "pipelineVersion": 1,
        "status": "PipelineTerminating",
        "lastChangeTimestamp": "2022-03-30T15:44:57.317913472Z"
      }
    },
    {
      "pipeline": {
        "name": "iris",
        "steps": [
          {
            "name": "iris"
          }
        ],
        "output": {
          "inputs": [
            "iris.outputs"
          ]
        },
        "kubernetesMeta": {
          
        }
      },
      "state": {
        "pipelineVersion": 2,
        "status": "PipelineCreating",
        "lastChangeTimestamp": "2022-03-30T15:45:08.545973059Z"
      }
    }
  ]
}


In [2]:
!curl -v http://${MESH_URL}/v2/models/iris/infer -H "Content-Type: application/json" -H "seldon-model: iris.pipeline"\
        -d '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}'

*   Trying 0.0.0.0...
* Connected to 0.0.0.0 (127.0.0.1) port 9000 (#0)








* upload completely sent off: 94 out of 94 bytes






* Closing connection 0


In [8]:
!grpcurl -d '{"name":"iris"}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/UnloadPipeline

{
  
}


In [3]:
!grpcurl -d '{"name":"iris","allVersions":true}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/PipelineStatus

{
  "pipelineName": "iris",
  "versions": [
    {
      "pipeline": {
        "name": "iris",
        "steps": [
          {
            "name": "iris"
          }
        ],
        "output": {
          "inputs": [
            "iris.outputs"
          ]
        },
        "kubernetesMeta": {
          
        }
      },
      "state": {
        "pipelineVersion": 1,
        "status": "PipelineReady",
        "reason": "removal requested",
        "lastChangeTimestamp": "2022-04-02T08:44:53.263695389Z"
      }
    },
    {
      "pipeline": {
        "name": "iris",
        "steps": [
          {
            "name": "iris"
          }
        ],
        "output": {
          "inputs": [
            "iris.outputs"
          ]
        },
        "kubernetesMeta": {
          
        }
      },
      "state": {
        "pipelineVersion": 2,
        "status": "PipelineCreating",
        "lastChangeTimestamp": "2022-04-02T08:45:24.824140452Z"
      }
    }
  ]
}


## K8s Pipeline Test

In [ ]:
!cat ../../samples/pipelines/example.yaml

In [ ]:
!kubectl create -f ../../samples/pipelines/example.yaml

In [ ]:
!curl -v http://${MESH_URL}/v2/models/iris/infer -H "Content-Type: application/json" -H "seldon-model: example.pipeline"\
        -d '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}'